In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.encoding.gen_encoding import GenerateEmbeddings

In [4]:
GenerateEmbeddings( dataset_path="data/annotation/scl/balanced.csv",
                    encoder_name="esm1_t34_670M_UR50S",
                    reset_param = False,
                    resample_param = False,
                    embed_batch_size = 128,
                    flatten_emb = "mean",
                    seq_start_idx = False,
                    seq_end_idx= False,
                    embed_folder = "embeddings",
                    # **encoder_params,
                    )

Generating esm1_t34_670M_UR50S upto 34 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/75 [00:00<?, ?it/s]

Overwritting embeddings/annotation/scl/balanced/esm1_t34_670M_UR50S/mean/train/embedding.h5


: 

: 

In [12]:
GenerateEmbeddings( dataset_path="data/proeng/thermo/mixed_split.csv",
                    encoder_name="esm1_t34_670M_UR50S",
                    reset_param = False,
                    resample_param = False,
                    embed_batch_size = 128,
                    flatten_emb = "mean",
                    seq_start_idx = False,
                    seq_end_idx= False,
                    embed_folder = "embeddings",
                    # **encoder_params,
                    )

In [8]:
# try load saved emb


In [9]:
gb1_emb = tables.open_file("embeddings/proeng/gb1/low_vs_high/esm1_t6_43M_UR50S/mean/test/embedding.h5")

In [11]:
gb1_emb.flush()
gb1_emb.root.layer0[0:5].shape

(5, 768)